In [1]:
from automl_surrogate.data.heterogeneous.dataset import HeteroPipelineDataset
from automl_surrogate.models.heterogeneous.listwise.pipelines_ranking import HeteroPipelineRankingSurrogateModel
import yaml

/home/cherniak/miniconda3/envs/gamlet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle
with open ("/home/cherniak/itmo_job/GAMLET/data/no_meta_features_and_fedot_pipelines_raw/id2pipeline_path.pickle", "rb") as f:
    id2pipe = pickle.load(f)
with open ("/home/cherniak/itmo_job/GAMLET/data/no_meta_features_and_fedot_pipelines_raw/id2dataset_id.pickle", "rb") as f:
    id2dataset = pickle.load(f)

ds = HeteroPipelineDataset(
    "/home/cherniak/itmo_job/GAMLET/data/no_meta_features_and_fedot_pipelines_raw/train_task_pipe_comb.csv",
    id2pipe,
    id2dataset,
    pipelines_per_step=10,
    use_dataset_with_id=27,
)

/home/cherniak/miniconda3/envs/gamlet/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(
/home/cherniak/miniconda3/envs/gamlet/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The identity link alias is deprecated. Use Identity instead. The identity link alias will be removed after the 0.15.0 release.
  warnings.warn(
/home/cherniak/miniconda3/envs/gamlet/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The inverse_power link alias is deprecated. Use InversePower instead. The inverse_power link alias will be removed after the 0.15.0 release.
  warnings.warn(
/home/cherniak/miniconda3/envs/gamlet/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The inverse_squared link alias is deprecated. Use InverseSquared instead. The

In [3]:
samples = [ds[i] for i in range(1024)]
batch = ds.collate_fn(samples)
x, y = batch

In [4]:
with open("/home/cherniak/itmo_job/surrogate/configs/mhsa_ranking_surrogate.yml") as f:
    config = yaml.safe_load(f)
model_config = {k: v for k, v in config["model"].items() if k != "class"}

In [5]:
import torch
model = HeteroPipelineRankingSurrogateModel(**model_config)
state_dict = torch.load(
    "/home/cherniak/itmo_job/surrogate/experiment_logs/no_meta_features_and_fedot_pipelines_(type_and_hparams)/train_node_embedder_from_scratch/overfit_27th_dataset/simple_graph_encoder/rank_over_10/mhsa/version_0/checkpoints/last.ckpt",
    map_location="cpu",
)["state_dict"]
model.load_state_dict(state_dict)
model = model.eval()

In [6]:
import torch
with torch.no_grad():
    scores = model.forward(x)

In [11]:
from scipy.stats import kendalltau
from sklearn.metrics import ndcg_score
from automl_surrogate.metrics import precision

kts = []
nscgs = []

for a, b in zip(y, scores):
    kts.append(kendalltau(a, b)[0])
    nscgs.append(ndcg_score(a.reshape(1, -1), b.reshape(1, -1)))

In [15]:
import numpy as np
np.mean(nscgs), np.mean(kts), precision(y, scores)

(0.9983641581398129, 0.9122491317199797, 0.690234363079071)